# Exploratory data analysis (EDA) of car data

## Libraries and settings

In [23]:
# Libraries
import os
import re
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime, timedelta

# seaborn graphics settings
sns.set_theme(color_codes=True)

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

u:\Lektionen\GitHub_Repositories\python_exploratory_data_analysis


## Importing and preparing the car data

### Importing the car data

In [24]:
# Columns to read
columns = ["Offer_Id",
            "Type",
            "Price",
            "HP",
            "Mileage",
            "Fuel_Type",
            "Transmission",
            "Dealer_PLZ",
            "Dealer_City",
            "Dealer_Street_House_Nr",
            "Init_Regist_MY",
            "Init_Regist_Month",
            "Init_Regist_Year",
            "Init_Regist_Dt"]

# Read and select variables
df_orig = pd.read_csv("autoscout24_data_enriched_cleaned.csv", sep=",")[columns]

# Remove duplicates
df_orig = df_orig.drop_duplicates()
df_orig.head(5)

# Remove missing values
df = df_orig.dropna()
df.head(5)

,Offer_Id,Type,Price,HP,Mileage,Fuel_Type,Transmission,Dealer_PLZ,Dealer_City,Dealer_Street_House_Nr,Init_Regist_MY,Init_Regist_Month,Init_Regist_Year,Init_Regist_Dt
0,7324420,AUDI A5 Sportback 3.0 TDI quattro S-tronic (Li...,22500,245,75000,Diesel,Automatisiertes Schaltgetriebe,8488,Turbenthal,Mettlenstrasse 3,10.2014,10.0,2014.0,2014-10
1,7512768,MERCEDES-BENZ SLK 200 7G-Tronic (Cabriolet),23749,184,46655,Benzin,Automat sequentiell,3186,Düdingen,Brugerastrasse 60,6.2013,6.0,2013.0,2013-06
2,7512034,MERCEDES-BENZ C 350 Avantgarde 4Matic 7G-Troni...,18500,306,138955,Benzin,Automat sequentiell,1262,Eysins,1262 Eysins,6.2011,6.0,2011.0,2011-06
3,7512728,MERCEDES-BENZ A 45 AMG 4Matic Speedshift 7G-DC...,36000,360,43000,Benzin,Automatisiertes Schaltgetriebe,4314,Zeiningen,Am Stutz 21,8.2015,8.0,2015.0,2015-08
4,7490242,AUDI A5 Sportback 2.0 TFSI Sport quattro S-tro...,48500,252,43300,Benzin,Automatisiertes Schaltgetriebe,3250,Lyss,3250 Lyss,9.2018,9.0,2018.0,2018-09


## Univariate non-graphical exploratory data analysis (EDA)

### Quantiles

### Shape (number of rows and colums)

### Data types

### Summary statistics of numeric variables

### Statistical measures (min, max, std, mean, median, count) for selected variables

### Skewness

### Kurtosis

### Extreme values

### Get a list of categories of categorical variable

## Multivariate non-graphical exploratory data analysis (EDA)

### Cross-tabulation

### Pivot tables

### Correlation matrix

### Covariance matrix

## Univariate graphical exploratory data analysis (EDA)

### Line chart (matplotlib)

### Boxplot (seaborn)

### Histogram  (matplotlib)

### Density plot  (seaborn)

### Quantile-Quantile (QQ-) plot

### Barchart  (matplotlib)

### Piechart (matplotlib)

## Multivariate graphical exploratory data analysis (EDA)

### Scatterplot (matplotlib)

### Scatterplot (matplotlib) with regression line

### Scatterplot-matrix (seaborn)

### Hexagonal binning plot  (matplotlib)

### Correlation heatmap (seaborn)

### Bubble plot (seaborn)

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [25]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
NT
Windows | 10
Datetime: 2024-09-28 19:38:53
Python Version: 3.11.9
-----------------------------------
